# Южный Парк

Импортирую всякие модули

In [86]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

from nltk import word_tokenize 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

Вытаскиваю и нормирую данные.

In [3]:
SO_PARK = pandas.read_csv('All-seasons.csv', sep=',')
SO_PARK.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


Под главными персонажами я поняла четвёрку главных.

In [4]:
stan = SO_PARK[SO_PARK['Character'] == 'Stan']
kyle = SO_PARK[SO_PARK['Character'] == 'Kyle']
eric = SO_PARK[SO_PARK['Character'] == 'Cartman']
kenny = SO_PARK[SO_PARK['Character'] == 'Kenny']

In [5]:
print('stan: ' + str(len(stan)))
print('kyle: ' + str(len(kyle)))
print('eric: ' + str(len(eric)))
print('kenny: ' + str(len(kenny)))

stan: 7680
kyle: 7099
eric: 9774
kenny: 881


Данные несбалансированы, потому что Кенни говорит слишком мало. При этом данных очень много, потому что это все сезоны. Поэтому я, пожалуй, просто выкину часть лишних данных.

In [9]:
less_stan = stan.sample(n=len(kenny))
less_kyle = kyle.sample(n=len(kenny))
less_eric = eric.sample(n=len(kenny))
balanced = pandas.concat([less_stan, less_kyle, less_eric, kenny])
balanced.groupby('Character').describe()

Episode        Line Season
Character                                  
Cartman   count      881         881    881
          unique      18         875     18
          top          4  Butters.\n      4
          freq        75           3     79
Kenny     count      881         881    881
          unique      18         754     18
          top          3   (Yeah!)\n      3
          freq       106          17    132
Kyle      count      881         881    881
          unique      18         843     18
          top          9     What?\n      2
          freq        79          13     97
Stan      count      881         881    881
          unique      18         853     18
          top         10     What?\n      2
          freq        75           8     95

Теперь датасет сбалансирован.

## Baseline

In [73]:
import random

rand_names = ['Cartman', 'Kenny', 'Kyle', 'Stan'] * 881
random.shuffle(rand_names)
print(rand_names[:100])

['Kenny', 'Kyle', 'Kenny', 'Stan', 'Kenny', 'Kenny', 'Kyle', 'Cartman', 'Kenny', 'Kenny', 'Stan', 'Kenny', 'Stan', 'Cartman', 'Stan', 'Kenny', 'Kenny', 'Cartman', 'Kenny', 'Kyle', 'Stan', 'Cartman', 'Cartman', 'Kenny', 'Kyle', 'Kyle', 'Kyle', 'Kenny', 'Stan', 'Cartman', 'Cartman', 'Kenny', 'Kenny', 'Stan', 'Kyle', 'Cartman', 'Stan', 'Cartman', 'Stan', 'Kyle', 'Stan', 'Cartman', 'Stan', 'Kenny', 'Cartman', 'Kyle', 'Kenny', 'Kyle', 'Stan', 'Stan', 'Stan', 'Kenny', 'Kenny', 'Stan', 'Kyle', 'Kyle', 'Kyle', 'Kyle', 'Stan', 'Cartman', 'Kenny', 'Kyle', 'Cartman', 'Kenny', 'Cartman', 'Kenny', 'Stan', 'Kyle', 'Kenny', 'Kyle', 'Cartman', 'Cartman', 'Kenny', 'Stan', 'Kyle', 'Kenny', 'Kyle', 'Stan', 'Kenny', 'Cartman', 'Kyle', 'Kyle', 'Kenny', 'Kenny', 'Kyle', 'Kyle', 'Stan', 'Cartman', 'Kyle', 'Kenny', 'Stan', 'Kenny', 'Kyle', 'Kenny', 'Kenny', 'Cartman', 'Stan', 'Stan', 'Cartman', 'Kenny']


In [74]:
print(classification_report(balanced['Character'], rand_names))

             precision    recall  f1-score   support

    Cartman       0.27      0.27      0.27       881
      Kenny       0.27      0.27      0.27       881
       Kyle       0.25      0.25      0.25       881
       Stan       0.25      0.25      0.25       881

avg / total       0.26      0.26      0.26      3524



Если распределять героев с рандомно с одинаковой вероятностью, точность в райное 0.25, что полностью соответствует теории вероятностей.

## Preparing data

Дальше я попробую лемматизировать и не лемматизировать. Лемматизировать – потому что это кажется разумным решением, в конце концов, о человеке должны больше говорять лексемы, которые он употребляет, чем словоформы. Не лемматизировать, потому что в прошлом дз в задаче классификации писем внезапно лучше показали семя не лемматизированные словари.

In [39]:
# не лемматизируя
bow_non_lem = CountVectorizer()
bow_non_lem.fit_transform(balanced['Line'])
bowed_non_lem = bow_non_lem.transform(balanced['Line'])

In [128]:
# функция для лемматизации. коммент: я пыталась сделать это как полагается, и сначала сделать POS-tagging,
# но лемматизатор ворднета не идеален. он, например, вообще не умеет лемматизировать местоимения...

from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

def lemmatize(word):
    noun = lmtzr.lemmatize(word)
    verb = lmtzr.lemmatize(word, 'v')
    if verb != word:
        return verb
    return noun

def tokenize_lemmatize(text):
    text = word_tokenize(text.lower())
    return [lemmatize(word) for word in text if word not in '!?,.":;']

bow_lem = CountVectorizer(tokenizer=tokenize_lemmatize)
bow_lem.fit_transform(balanced['Line'])
bowed_lem = bow_lem.transform(balanced['Line'])

In [ ]:
Смотрю на частотные словари.

In [131]:
d = bow_non_lem.vocabulary_
for key in d:
    d[key] = int(d[key])
sorted(d, key=lambda x: -d[x])

['zombies',
 'ziplining',
 'zillion',
 'zero',
 'zeit',
 'yup',
 'yummy',
 'yuh',
 'yow',
 'youuu',
 'youtube',
 'youth',
 'yourself',
 'yours',
 'your',
 'younger',
 'young',
 'you',
 'yo',
 'yikes',
 'yeti',
 'yet',
 'yesterday',
 'yes',
 'yer',
 'yep',
 'yek',
 'yehes',
 'yeh',
 'yeessss',
 'yeeeeehah',
 'yeeaahh',
 'yee',
 'years',
 'year',
 'yeahyeah',
 'yeahhh',
 'yeahah',
 'yeah',
 'ye',
 'yay',
 'yahtzee',
 'yahoo',
 'yaaayaya',
 'ya',
 'xbox',
 'wählen',
 'wwe',
 'wway',
 'wussy',
 'wuss',
 'wull',
 'wuh',
 'wugh',
 'wth',
 'wrote',
 'wrong',
 'write',
 'wrestling',
 'wrestlin',
 'wrestle',
 'wrath',
 'wrapped',
 'wow',
 'wouldn',
 'would',
 'worthless',
 'worth',
 'worshiping',
 'worse',
 'worry',
 'worried',
 'wormhole',
 'worm',
 'worldwide',
 'world',
 'works',
 'working',
 'workin',
 'worked',
 'work',
 'words',
 'word',
 'wooooooooo',
 'wooooooo',
 'woooooh',
 'woooohoooo',
 'woooohoohoo',
 'wooo',
 'woohoooo',
 'woohooo',
 'woohoohoo',
 'woohoo',
 'wood',
 'woo',
 'wond

In [136]:
d = bow_lem.vocabulary_
for key in d:
    d[key] = int(d[key])
sorted(d, key=lambda x: -d[x])

['—christmastime',
 'zombie',
 'ziplining',
 'zillion',
 'zero',
 'zeit',
 'yup',
 'yummy',
 'yuh',
 'yow',
 'youuu',
 'youtube',
 'youth',
 'yourself',
 'yours',
 'your',
 'younger',
 'young',
 'you-',
 "you're-you",
 'you',
 'yo',
 'yeti',
 'yet',
 'yes…',
 'yesterday',
 'yes',
 'yer',
 'yep',
 'yehes',
 'yeh-heah',
 'yeh-hah',
 'yeessss',
 'yeeeeehah',
 'yeeaahh',
 'yee',
 'year',
 'yeahyeah',
 'yeahhh',
 'yeahah',
 'yeah',
 'ye-eah',
 'yay',
 'yahtzee',
 'yahoo',
 'yaaayaya',
 'ya',
 'y-yeah',
 'xbox',
 'x',
 'wählen',
 'wwe',
 'wway',
 'wussy',
 'wuss',
 'wull',
 'wuh',
 'wugh',
 'wth',
 'wrong',
 'write',
 'wrestlin',
 'wrestle',
 'wrath',
 'wrap',
 'wow',
 'would',
 'worthless',
 'worth',
 'worse',
 'worry',
 'wormhole',
 'worm',
 'worldwide',
 'world',
 'workin',
 'work',
 'word',
 'wooooooooo',
 'wooooooo',
 'woooooh',
 'woooohoooo',
 'woooohoohoo',
 'wooo',
 'woohoooo',
 'woohooo',
 'woohoohoo',
 'woohoo',
 'wood',
 'woo-hoo-hoo',
 'woo-hoo',
 'woo',
 'wondrous',
 'wonder',
 

In [126]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

## Training and testing

In [132]:
# (криво) лематизированные реплики
lem_train, lem_test, y_train_lem, y_test_lem = train_test_split(bowed_lem, 
                                                    balanced['Character'], 
                                                    test_size=0.2)
print(lem_train.shape, lem_test.shape, y_train_lem.shape, y_test_lem.shape)

(2819, 3422) (705, 3422) (2819,) (705,)


In [133]:
# не лемматизированные реплики
nlem_train, nlem_test, y_train_nlem, y_test_nlem = train_test_split(bowed_non_lem, 
                                                    balanced['Character'], 
                                                    test_size=0.2)
print(nlem_train.shape, nlem_test.shape, y_train_nlem.shape, y_test_nlem.shape)

(2819, 3799) (705, 3799) (2819,) (705,)


### Рандомный лес

In [137]:
# на лемматизированных данных
model = RandomForestClassifier(n_estimators = 100)
model.fit(lem_train, y_train_lem)
predict_y = model.predict(lem_test)
print(classification_report(y_test_lem, predict_y))

             precision    recall  f1-score   support

    Cartman       0.44      0.45      0.44       168
      Kenny       0.53      0.59      0.56       185
       Kyle       0.32      0.34      0.33       165
       Stan       0.41      0.34      0.37       187

avg / total       0.43      0.43      0.43       705



In [124]:
# на нелемматизированных данных
model = RandomForestClassifier(n_estimators = 100)
model.fit(nlem_train, y_train_nlem)
predict_y = model.predict(nlem_test)
print(classification_report(y_test_nlem, predict_y))

             precision    recall  f1-score   support

    Cartman       0.37      0.33      0.35       155
      Kenny       0.45      0.60      0.52       184
       Kyle       0.33      0.32      0.32       186
       Stan       0.35      0.29      0.32       180

avg / total       0.38      0.39      0.38       705



### Наивный Байес 

## Testing

In [101]:
predict_y = model.predict(lem_test)
print(classification_report(y_test_lem, predict_y))

             precision    recall  f1-score   support

    Cartman       0.47      0.47      0.47       154
      Kenny       1.00      0.94      0.97       199
       Kyle       0.41      0.45      0.43       180
       Stan       0.38      0.36      0.37       172

avg / total       0.58      0.57      0.58       705



In [103]:
predict_y = model.predict(nlem_test)
print(classification_report(y_test_nlem, predict_y))

             precision    recall  f1-score   support

    Cartman       0.37      0.31      0.34       155
      Kenny       0.45      0.61      0.52       184
       Kyle       0.35      0.30      0.32       186
       Stan       0.31      0.28      0.30       180

avg / total       0.37      0.38      0.37       705



## Graphs